In [2]:
import numpy as np
import math

In [3]:
L, d_k, d_v = 4, 8, 8
# L - length of input sequence

# Query Vector for what i am looking for?
q = np.random.randn(L, d_k) 

# key Vector for what i can offer?
k = np.random.randn(L, d_k)

# Value Vector for what i can actually offer?
v = np.random.randn(L, d_k)

In [4]:
print("\nQ: ", q)
print("\nK: ", k)
print("\nV: ", v)


Q:  [[ 0.35292165 -2.42305817  0.69046244 -0.47845401  0.50689946 -1.02207339
  -0.84721908 -1.41738232]
 [-0.39111263 -1.22635155 -0.25379764  0.94699906 -0.04908601  0.59878926
   0.09471052 -1.18064159]
 [-1.29704503 -1.56886625  0.03308404  0.67139391  0.01403374  1.89264955
   1.24032343 -0.69794846]
 [ 0.65181004  0.70532573 -0.75087357 -2.1500712   0.35381951 -0.32147255
  -0.18700822  1.05418744]]

K:  [[-1.05556466 -0.65575548 -0.60045189  0.1701294  -1.6596807  -0.39826288
   0.21002508 -0.22950192]
 [-0.37452831 -0.97842119  0.75160694 -0.57925442  1.18437778 -1.38003843
   1.04940125  1.10597404]
 [ 1.90459612 -0.87242125 -0.21312456 -0.68596798  0.54010164 -0.43779328
  -1.92585614  0.17720304]
 [ 0.26655539 -0.28642872  0.81852355  0.58848341  1.22661266 -0.90873139
  -0.03056235  0.92031232]]

V:  [[-1.75679877  2.34325231  0.1777176   1.95375292  1.31817722  0.57872102
  -0.87563972 -0.11241762]
 [ 1.49006228 -0.83211621 -2.04190506 -0.318155    1.47569886 -1.05394128


# Self Attention

In [12]:
np.matmul(q, k.T)

array([[ 0.43353084,  2.58889548,  5.06884197,  1.3437222 ],
       [ 1.66437974, -1.48379655, -0.95080644, -1.09723868],
       [ 2.13588196, -0.4088734 , -4.90261313, -1.85711561],
       [-1.80588701,  1.57920722,  3.13979647, -0.20614222]])

In [14]:
q.var(), k.var(), np.matmul(q, k.T).var()

(0.9842740304810997, 0.8086087934326633, 5.476324364169954)

In [15]:
scaled = np.matmul(q, k.T)/math.sqrt(d_k)
q.var(), k.var(), scaled.var()

(0.9842740304810997, 0.8086087934326633, 0.6845405455212442)

# Maksing

specifically require in decoder part of transformer so we dont look at the future word when trying to generate current context

In [8]:
mask = np.tril(np.ones((L, L)))
mask

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

In [9]:
mask[mask==0] = -np.infty
mask[mask==1] = 0

In [10]:
mask

array([[  0., -inf, -inf, -inf],
       [  0.,   0., -inf, -inf],
       [  0.,   0.,   0., -inf],
       [  0.,   0.,   0.,   0.]])

In [16]:
scaled

array([[ 0.1532763 ,  0.91531278,  1.79210627,  0.47507754],
       [ 0.5884471 , -0.5246013 , -0.33616084, -0.38793246],
       [ 0.75514831, -0.14455858, -1.7333355 , -0.65658952],
       [-0.63847748,  0.55833407,  1.11008569, -0.07288228]])

In [17]:
scaled + mask

array([[ 0.1532763 ,        -inf,        -inf,        -inf],
       [ 0.5884471 , -0.5246013 ,        -inf,        -inf],
       [ 0.75514831, -0.14455858, -1.7333355 ,        -inf],
       [-0.63847748,  0.55833407,  1.11008569, -0.07288228]])

# Softmax

operation used to convert vectors into a probability distribution

In [29]:
def softmax(x):
    return (np.exp(x).T / np.sum(np.exp(x), axis=-1)).T

def scaled_dot_product_attention(q, k, v, mask=None):
    d_k = q.shape[-1]
    scaled = np.matmul(q, k.T) / math.sqrt(d_k)
    if mask is not None:
        scaled = scaled+mask
    attention = softmax(scaled)
    out = np.matmul(attention, v)
    return out, attention

In [31]:
values , attention = scaled_dot_product_attention(q, k, v, mask=mask)
print("Q:\n",q)
print("K:\n",k)
print("V:\n",v)
print("Values:\n",values)
print("Attention:\n",attention)


Q:
 [[ 0.35292165 -2.42305817  0.69046244 -0.47845401  0.50689946 -1.02207339
  -0.84721908 -1.41738232]
 [-0.39111263 -1.22635155 -0.25379764  0.94699906 -0.04908601  0.59878926
   0.09471052 -1.18064159]
 [-1.29704503 -1.56886625  0.03308404  0.67139391  0.01403374  1.89264955
   1.24032343 -0.69794846]
 [ 0.65181004  0.70532573 -0.75087357 -2.1500712   0.35381951 -0.32147255
  -0.18700822  1.05418744]]
K:
 [[-1.05556466 -0.65575548 -0.60045189  0.1701294  -1.6596807  -0.39826288
   0.21002508 -0.22950192]
 [-0.37452831 -0.97842119  0.75160694 -0.57925442  1.18437778 -1.38003843
   1.04940125  1.10597404]
 [ 1.90459612 -0.87242125 -0.21312456 -0.68596798  0.54010164 -0.43779328
  -1.92585614  0.17720304]
 [ 0.26655539 -0.28642872  0.81852355  0.58848341  1.22661266 -0.90873139
  -0.03056235  0.92031232]]
V:
 [[-1.75679877  2.34325231  0.1777176   1.95375292  1.31817722  0.57872102
  -0.87563972 -0.11241762]
 [ 1.49006228 -0.83211621 -2.04190506 -0.318155    1.47569886 -1.05394128
   

In [20]:
attention

array([[1.        , 0.        , 0.        , 0.        ],
       [0.75269699, 0.24730301, 0.        , 0.        ],
       [0.671265  , 0.27299599, 0.05573901, 0.        ],
       [0.08462826, 0.28008128, 0.48630279, 0.14898766]])

In [22]:
#multiplying attention matrix to value matrix
new_v = np.matmul(attention, v)
new_v

array([[-1.75679877,  2.34325231,  0.1777176 ,  1.95375292,  1.31817722,
         0.57872102, -0.87563972, -0.11241762],
       [-0.95384026,  1.55797412, -0.37120176,  1.39190325,  1.3571328 ,
         0.17495872, -0.60032228,  0.18538817],
       [-0.72406995,  1.46536986, -0.48829051,  1.28230377,  1.24907795,
         0.02655701, -0.46089191,  0.17110361],
       [ 0.69232178,  0.78874585, -1.16832123,  0.79428966,  0.45985458,
        -0.95509522,  0.34981046, -0.09791819]])

In [23]:
v

array([[-1.75679877,  2.34325231,  0.1777176 ,  1.95375292,  1.31817722,
         0.57872102, -0.87563972, -0.11241762],
       [ 1.49006228, -0.83211621, -2.04190506, -0.318155  ,  1.47569886,
        -1.05394128,  0.23764006,  1.09179655],
       [ 0.86880906,  2.14555269, -0.89980479,  1.03469467, -0.69301851,
        -1.33113993,  1.11267869, -0.92377774],
       [ 0.00774416, -1.47587808, -1.16710467,  1.44227573,  1.82565956,
        -0.41308845, -1.23327636,  0.36942283]])